In [1]:
 !pip install PyMySQL

In [31]:
import pymysql
blogdb = pymysql.connect(host='blogdb.cm2yxwfja9ii.ap-northeast-2.rds.amazonaws.com',
                      user='admin',
                      password='                      database='blogdb',
                      charset='utf8',
                      port=3306)

cursor =blogdb.cursor(pymysql.cursors.DictCursor)

# 데이터불러오기
query = "SELECT content FROM post_data where map=1 And ad_status = 'NON-AD' and res_id =1"
cursor.execute(query)
results = cursor.fetchall()

In [9]:
text1 = results[0]['content']
text1

'잠실맛집  잠실라멘  송리단길맛집  류센소  잠실맛집  잠실라멘  송리단길맛집  류센소   부산에서 라멘 맛집으로 웨이팅 할 만큼 유명한 류센소가 송리단길에도 오픈했다는 소식에 얼마나 맛있을지 궁금해 한걸음에 달려와봤다    송리단길에 이미 잠실 라멘 맛집으로 유명한 매장이 많아 웨이팅 없이 바로 입장 가능했다  잠실 라멘 맛집은 지하철 9호선 송파나루역 1번 출구에서 도보 6분이 걸리고 지하철 2 8호선 잠실역 10번 출구에서 도보 16분이다  올 때 편한 역을 이용하면 된다    송리단길 맛집 중 주차 가능한 매장은 없다고 보면 되는데 매장 앞에 주차도 2대 가능하다  부산에서 줄 서서 먹어야 될 만큼  라멘 성지 로 불리는 류센소가 송리단길에 오픈했다  배고픈 나머지 앉자마자 메뉴판을 봤다    보통 류센소 라멘을 많이들 먹어서 라멘은 금방 골랐는데 사이드 메뉴는 교자  토리 가라아게  문어 가라아게 중 뭐 먹어야 될지 한참동안 고민됐다    가격적인 면에서 토리 가라아게가 가성비가 뛰어나 보여 류센소 라멘과 가라아게로 주문  메뉴판을 보면 주문 안내사항이랑 먹는 방법이 설명되어 있어서 먹을 때 참고  보통 라멘집은 아담한 규모인데 송리단길 라멘 맛집은 넓은 규모에 쾌적한 공간으로 보다 편안하게 맛있는 식사를 할 수 있어 보였다  좌석은 일반 테이블이랑 바 테이블로 나누어져 있었고 편한 자리로 골라 앉으면 된다    우리는 일본 여행이라도 온 분위기를 내고 싶어서 고민없이 바 테이블에 앉았다  일본스러운 느낌을 내고 싶으면 바 테이블을 이길 수 없다  다행히 바 테이블 간격이 넓어서 불편하지 않았다  톤 다운된 우드로 제작한 바 테이블이 분위기를 차분함과 동시에 멋스러워 보이도록 만들었다  류센소 라멘은 기본적으로 국물이 깔끔하고 담백해 테이블에 준비된 다진 마늘과 후추를 취향에 따라 넣어 먹으면 색다른 국물을 맛볼 수 있다    개인적으로 다진 마늘을 넣었을 때 국물의 깊이가 달라져서 먹다가 꼭 넣어 먹는 걸 추천한다  테이블마다 갓 장아찌  생강 초

## 메뉴 분석

In [ ]:
import pymysql
blogdb = pymysql.connect(host='blogdb.cm2yxwfja9ii.ap-northeast-2.rds.amazonaws.com',
                      user='admin',
                      password='                      database='blogdb',
                      charset='utf8',
                      port=3306)

cursor =blogdb.cursor(pymysql.cursors.DictCursor)

# 데이터불러오기
query = "SELECT content,menu_count FROM post where map=1 And ad_status = 'NON-AD' and res_id = "
cursor.execute(query)
results = cursor.fetchall()

In [28]:
import re

menu_keywords = ['류센소', '류센소카라', '류센소카키','류센소아사리', '교자', '토리카라아게', '카키후라이', '문어카라아게', '생맥주', '하이볼','가라아게']

menu_counts = {keyword: 0 for keyword in menu_keywords}  # 메뉴별 언급 횟수 초기화

for result in results:
    text = result['content']
    menu_matches = re.findall('|'.join(menu_keywords), text)

    for match in menu_matches:
        menu_counts[match] += 1

for menu, count in menu_counts.items():
    print(f"{menu}: {count}")

류센소: 423
류센소카라: 0
류센소카키: 0
류센소아사리: 0
교자: 53
토리카라아게: 2
카키후라이: 10
문어카라아게: 0
생맥주: 25
하이볼: 64
가라아게: 46


In [ ]:
for menu, count in menu_counts.items():
    query = "INSERT INTO menu_count (menu, count) VALUES (%s, %s)"
    values = (menu, count)
    cursor.execute(query, values)

blogdb.commit()
cursor.close()

## 키워드분석

In [21]:
import pymysql
blogdb = pymysql.connect(host='blogdb.cm2yxwfja9ii.ap-northeast-2.rds.amazonaws.com',
                      user='admin',
                      password='                      database='blogdb',
                      charset='utf8',
                      port=3306)
cursor =blogdb.cursor(pymysql.cursors.DictCursor)


# 데이터불러오기
query1 = "SELECT title,content FROM post_data where ad_status='NON-AD' and map=1 and res_id <11"
cursor.execute(query1)
results1 = cursor.fetchall()

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# 'title' 데이터 추출
titles = [item['title'] for item in results1]

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=1000)  # 원하는 단어 수로 조정
title_vectors = vectorizer.fit_transform(titles)

# K-means 클러스터링
num_clusters = 1  # 클러스터 개수
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(title_vectors)

# 클러스터 내 데이터의 비중 계산
cluster_weights = kmeans.transform(title_vectors)

# 클러스터 중심 단어 및 비중 출력
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

# 클러스터 내 언급 횟수 계산
order_labels = kmeans.labels_
cluster_counts = {i: {} for i in range(num_clusters)}

for i, label in enumerate(order_labels):
    for keyword in terms:
        if keyword in titles[i]:
            cluster_counts[label][keyword] = cluster_counts[label].get(keyword, 0) + 1

# 결과 출력
for i in range(num_clusters):
    print(f"Cluster {i + 1}:")
    cluster_keywords = [terms[ind] for ind in order_centroids[i, :10]]
    cluster_weight = cluster_weights[:, i]
    cluster_count = cluster_counts[i]

    for keyword, weight in zip(cluster_keywords, cluster_weight):
        count = cluster_count.get(keyword, 0)
        print(f"{keyword} (언급 횟수: {count}, 비중: {weight:.3f})")

Cluster 1:
송리단길 (언급 횟수: 80, 비중: 0.879)
류센소 (언급 횟수: 48, 비중: 0.939)
잠실 (언급 횟수: 62, 비중: 0.936)
맛집 (언급 횟수: 58, 비중: 0.948)
란슬롯 (언급 횟수: 50, 비중: 0.942)
와인바 (언급 횟수: 38, 비중: 0.999)
송리단 (언급 횟수: 106, 비중: 0.941)
송리단길점 (언급 횟수: 25, 비중: 0.982)
케이쿠키 (언급 횟수: 31, 비중: 0.883)
라멘 (언급 횟수: 29, 비중: 0.951)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## 포스팅 날짜 기준 포스팅 수

In [1]:
import pymysql
blogdb = pymysql.connect(host='blogdb.cm2yxwfja9ii.ap-northeast-2.rds.amazonaws.com',
                      user='admin',
                      password='                      database='blogdb',
                      charset='utf8',
                      port=3306)
cursor =blogdb.cursor(pymysql.cursors.DictCursor)


# 데이터불러오기
query2 = "SELECT post_date FROM post_data where ad_status='NON-AD' and map=1 and res_id <1"
cursor.execute(query2)
result2 = cursor.fetchall()

In [3]:
type(result2)

tuple